reference https://stackoverflow.com/questions/62346965/input-pipeline-w-keras-utils-sequence-object-or-tf-data-dataset  
Ingest Sequence-based generator with the tf.data.Dataset.from_generator() method  

In [1]:
import sys
# print(sys.path)
# sys.path.append('.')
# print(sys.path)
import tensorflow as tf
from config import config
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import math

In [2]:
from tensorflow.keras.utils import Sequence

**Sequence:**    (**Is this similar to Pytorch's torch.data.Dataset ??** )    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Base object for fitting to a sequence of data, such as a dataset.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Every Sequence must implement the __getitem__ and the __len__ methods.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If you want to modify your dataset between epochs you may implement **on_epoch_end**.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The method __getitem__ should return a complete batch.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Sequence are a safer way to do **multiprocessing**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;

In [6]:
class YoloSequence(Sequence):
    def __init__(self,config_file,training=True):
        self.DATA_LOCATION=config_file["DATA_LOCATION"]
        self.META_DATA_CSV=config_file["META_DATA_CSV"]  #csv file holding information about the images. centers of bboxes, width etc.
        self.TRAIN_IMAGES_DIR=config_file["TRAIN_IMAGES_DIR"]  #directory holding training images
#         self.TEST_IMAGES_DIR=config_file["TEST_IMAGES_DIR"]  #directory holding test images
        self.EVAL_IMAGES_DIR=config_file["EVAL_IMAGES_DIR"]  #directory holding training images
        self.IMAGE_WIDTH=config_file["IMAGE_WIDTH"]      #image width input to yolo
        self.IMAGE_HEIGHT=config_file["IMAGE_HEIGHT"]    #image height input to yolo
        self.Y_GRIDS=config_file["Y_GRIDS"]              #number of grids in the y direction
        self.X_GRIDS=config_file["X_GRIDS"]              #number of grids in the x direction
        self.BBOXES=config_file["BBOXES"]                #bounding boxes per grid location
        self.nclasses=config_file["nclasses"]            #classes
        
        self.training=training                          #if this is  training or evaluation dataset
        
        self.grid_width=1/self.X_GRIDS            #since we use YOLO , the image width and height are normalized to 1. i.e IMAGE_WIDTH<==>1  . scaling . when not normalized grid_width=Image_width/no_of_xgrids
        self.grid_height=1/self.Y_GRIDS           #since we use YOLO , the image width and height are normalized to 1. i.e IMAGE_HEIGHT<==>1  . scaling . when not normalized grid_height=Image_height/no_of_ygrids
        
        #using a prob_dist to balance 
        self.prob_dist_for_balancing=config_file["balance_prob"] 
        
        self.meta_df=pd.read_csv(os.path.join(self.DATA_LOCATION,self.META_DATA_CSV))
           
        self.image_dir=os.path.join(self.DATA_LOCATION,self.TRAIN_IMAGES_DIR)
        if not training:           
            self.image_dir=os.path.join(self.DATA_LOCATION,self.EVAL_IMAGES_DIR)
        self.batch_size=config_file["batch_size"]
        
        #list of all files in the images dir
        self.filelist=[f for f in listdir(self.image_dir) if isfile(join(self.image_dir, f))]
        self.df=pd.read_csv(os.path.join(self.DATA_LOCATION,self.META_DATA_CSV))                
        print(self.df.head())
        pass
    
    def __getitem__(self,idx):
        """
        Gets batch at position index idx
        """
        #read a batch_size worth of images. 
        #create a batch_size worth of labels
        batch_size=self.batch_size
        
        batch_x=np.zeros((batch_size,self.IMAGE_HEIGHT,self.IMAGE_WIDTH,3))                           #each batch holds an image. INPUT batch
        batch_y=np.zeros((batch_size,self.Y_GRIDS,self.X_GRIDS,5*self.BBOXES+self.nclasses))        #each bbox is described using 5 things. x,y,w,h,P(objectedness). LABELS 
        i=0
        
        for f in self.filelist[idx*batch_size:(idx+1)*batch_size]:
            print(f,f[0:-4])
#             idx_
            batch_x[i]=self.parse_image(os.path.join(self.image_dir,f))
            i=i+1    
            print(i,' / ',batch_size)
            
            label=np.zeros((self.Y_GRIDS,self.X_GRIDS,(5*self.BBOXES+self.nclasses)))
            
            #get the number of instances present in a single image file. We have this in the pandas df
            df_instances=self.df[self.df["Image_ID"]==f[0:-4]]
            
            for id in range(0,len(df_instances)):
                instance=df_instances.iloc[id]
               
                bboxid=id
                xbbox=instance['xc_yolo']
                ybbox=instance['yc_yolo']
                wbbox=instance['width_yolo']
                hbbox=instance['height_yolo']
                print(xbbox,ybbox,wbbox,hbbox)
                
                #calculate gridx,gridy  (i.e. find the grid in which (xbbox,ybbox) falls. This is the cell responsible for the bbox of this instance in the Image
                
                gridx=int(xbbox//self.grid_width)
                gridy=int(ybbox//self.grid_height)
                print("==>",gridy,gridx)
                #[gridy,gridx,0] holds xg inside grid
                label[gridy,gridx,5*bboxid+0]=xbbox%self.grid_width
                
                #[gridy,gridx,1] holds yg inside grid
                label[gridy,gridx,5*bboxid+1]=ybbox%self.grid_height
                
                #[gridy,gridx,2] holds gw. i.e width in grid units
                wg=wbbox/self.grid_width
                label[gridy,gridx,5*bboxid+2]=wg
                
                #[gridy,gridx,3] holds gh. i.e height in grid units
                hg=hbbox/self.grid_height
                label[gridy,gridx,5*bboxid+3]=hg
                
                #box of width=ceil(wbbox/gw) and height =ceil(hbbox/gh) centered at (gridx,gridy) holds objectedness. i.e 
                objectd_centre_x=gridx
                objectd_centre_y=gridy
                
                width_limits=[math.ceil(objectd_centre_x-wg/2),math.ceil(objectd_centre_x+wg/2)]
                print("WIDTH LIMITS ",width_limits)
                height_limits=[math.ceil(objectd_centre_y-hg/2),math.ceil(objectd_centre_y+hg/2)]
                print("HEIGHT LIMITS ",height_limits)

                if (height_limits[1]>self.Y_GRIDS):
                    height_limits[1]=self.Y_GRIDS
                if (width_limits[1]>self.X_GRIDS):
                    width_limits[1]=self.X_GRIDS
                #box of width=ceil(wbbox/gw) and height =ceil(hbbox/gh) centered at (gridx,gridy) holds objectedness at channel 4
                label[height_limits[0]:height_limits[1]+1,width_limits[0]:width_limits[1]+1,5*bboxid+4]=1.0
                
                #if class=="fruit_woodiness" then channel=5, class=="fruit_brownspot" then channel=6,  class=="fruit_healthy" then channel=7
                # the map of channel verus class is not terribly important. we could well have  assigned class=="fruit_woodiness" then channel=7 and so on.
                
                instance_class=instance['class']
                print("CLASS: ",instance_class)
                
                channel=""
                if instance_class=="fruit_woodiness":
                    channel=-1
                
                if instance_class=="fruit_brownspot":
                    channel=-2
                
                if instance_class=="fruit_healthy":
                    channel=-3
                
                
                #box of width=ceil(wbbox/gw) and height =ceil(hbbox/gh) centered at (gridx,gridy) holds objectedness at channel assigned by the above map.
                label[height_limits[0]:height_limits[1]+1,width_limits[0]:width_limits[1]+1,channel]=1.0
                

                
                
                
                
            #now for class information
            self.numpy_print(label)
                #label
                
                
#             label[]
            
            
            
         
        
        print("Before: ",type(batch_x))
        
        batch_x=tf.convert_to_tensor(batch_x)
        batch_y=tf.convert_to_tensor(batch_y)
        
        print("After: ",type(batch_x))
        return batch_x,batch_y
    
    def __len__(self):
        #length of data
        return 
    
    def parse_image(self,filepath):                                                   #change this as per project. Here everything's on file. so we read from file. different for streaming etc.
        image=tf.io.read_file(filepath)
        image = tf.image.decode_jpeg(image)
        image = tf.image.convert_image_dtype(image, tf.float32)
        return image
    
    def numpy_print(self,arr):
        large_width = 400
        np.set_printoptions(linewidth=large_width)
        np.set_printoptions(precision=3)
        for i in range(0,5*self.BBOXES+self.nclasses):
            print(arr[:,:,i])
        

In [7]:
config

{'DATA_LOCATION': '/data/kaggle/zindi/fruit_disease_detection',
 'META_DATA_CSV': 'train_yolo.csv',
 'TRAIN_IMAGES_DIR': 'Train_Images',
 'EVAL_IMAGES_DIR': 'Eval_Images',
 'IMAGE_WIDTH': 512,
 'IMAGE_HEIGHT': 512,
 'Y_GRIDS': 7,
 'X_GRIDS': 7,
 'BBOXES': 2,
 'nclasses': 3,
 'balance_prob': '[]',
 'batch_size': 4}

In [8]:
yseq=YoloSequence(config_file=config)

   Unnamed: 0     Image_ID            class   xmin   ymin  width  height  \
0           0  ID_007FAIEI  fruit_woodiness   87.0   87.5  228.0   311.0   
1           1  ID_00G8K1V3  fruit_brownspot   97.5   17.5  245.0   354.5   
2           2  ID_00WROUT9  fruit_brownspot  156.5  209.5  248.0   302.5   
3           3  ID_00ZJEEK3    fruit_healthy  125.0  193.0  254.5   217.0   
4           4  ID_018UIENR  fruit_brownspot   79.5  232.5  233.5   182.0   

    xc_yolo   yc_yolo  width_yolo  height_yolo  
0  0.392578  0.474609    0.445312     0.607422  
1  0.429688  0.380371    0.478516     0.692383  
2  0.547852  0.704590    0.484375     0.590820  
3  0.492676  0.588867    0.497070     0.423828  
4  0.383301  0.631836    0.456055     0.355469  


In [10]:
yseq[91]

ID_RYW9N4UI.jpg ID_RYW9N4UI
1  /  4
0.36865234375 0.48046875 0.7314453125 0.564453125
==> 3 2
WIDTH LIMITS  [0, 5]
HEIGHT LIMITS  [2, 5]
CLASS:  fruit_brownspot
[[0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.083 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]]
[[0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.052 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]]
[[0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   5.12 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   

(<tf.Tensor: shape=(4, 512, 512, 3), dtype=float64, numpy=
 array([[[[0.325, 0.271, 0.227],
          [0.322, 0.267, 0.224],
          [0.329, 0.282, 0.227],
          ...,
          [1.   , 1.   , 1.   ],
          [1.   , 1.   , 1.   ],
          [1.   , 1.   , 1.   ]],
 
         [[0.353, 0.298, 0.255],
          [0.345, 0.298, 0.251],
          [0.361, 0.314, 0.259],
          ...,
          [1.   , 1.   , 1.   ],
          [1.   , 1.   , 1.   ],
          [1.   , 1.   , 1.   ]],
 
         [[0.361, 0.314, 0.267],
          [0.361, 0.314, 0.267],
          [0.376, 0.329, 0.282],
          ...,
          [1.   , 1.   , 1.   ],
          [1.   , 1.   , 1.   ],
          [1.   , 1.   , 1.   ]],
 
         ...,
 
         [[0.173, 0.286, 0.075],
          [0.176, 0.286, 0.086],
          [0.176, 0.286, 0.094],
          ...,
          [0.588, 0.635, 0.596],
          [0.565, 0.612, 0.565],
          [0.537, 0.592, 0.541]],
 
         [[0.192, 0.298, 0.09 ],
          [0.184, 0.29 , 0.0

ID_EQJ63018.jpg  
ID_21VUG5GR.jpg  
ID_1V75KS83.jpg  
ID_TAXKFJL2.jpg  
ID_M4F97PA9.jpg  
ID_SV2FBE1B.jpg  
ID_O3XM1GUV.jpg  
ID_4PGDD6IU.jpg  
ID_UQ8W9NEP.jpg  
ID_P4YR8CCG.jpg  
ID_AYE5F0MN.jpg  
ID_64S2GR62.jpg  